In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import string
import os
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torch.autograd import Variable
from torchvision import transforms
from PIL import Image

In [2]:
use_cuda = torch.cuda.is_available()

print(use_cuda)

True


In [3]:
# classes' labels
sign_labels = ['A', 'B', 'C', 'D', 'E', 'F', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'T', 'U', 'V', 'W', 'X', 'Y']  
#non-static signs excluded 

dict_alph = {
   'a' : 0,
   'b' : 1,
   'c' : 2,
   'd' : 3,
   'e' : 4,
   'f' : 5,
   'h' : 6,
   'i' : 7,
   'k' : 8,
   'l' : 9,
   'm' : 10,
   'n' : 11,
   'o' : 12,
   'p' : 13,
   'q' : 14,
   'r' : 15,
   't' : 16,
   'u' : 17,
   'v' : 18,
   'w' : 19,
   'x' : 20,
   'y' : 21
}

# size of our images.
img_width, img_height = 64, 64

In [4]:
### Dataset folder (images must be stored in sub-folders per letter)

DATASET_FOLDER= "./DatasetLIS/dataset1" 

#partition folders 
train_data_dir = DATASET_FOLDER + "/train"
validation_data_dir = DATASET_FOLDER + "/validation"
testing_data_dir = DATASET_FOLDER + "/testing"

In [16]:
###  Epochs and batch-size definition  
epochs = 60
batch_size = 64
# input_shape = (3,64,64)

In [6]:
# from model.model_def_v2 import MyModel
from model.model_def_v1 import SignLanguageCNN
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    model = SignLanguageCNN().cuda()
else:
    model = SignLanguageCNN()

In [7]:
# summary(model) #takes the model and the input tensor shape, displays the output shape

In [8]:
from torchvision.io import read_image
from torchvision import transforms

def get_dataset(path_directory):

   transform = transforms.Compose([ transforms.Grayscale(),
                                    transforms.Resize((64, 64)),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.RandomRotation(15),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.485,), (0.229,)) ]) # @TODO: compute actual norm and devstd of the dataset
    
   print(path_directory)
   data = []
   labels = []
   img_train = []

   for folder in os.listdir(path_directory):
      subpath = path_directory+'/'+folder

      for img_name in os.listdir(subpath):
         #print(img_name)
         #im = read_image(subpath + '/'+ img_name)
         im = transform(Image.open(subpath + '/'+ img_name))
         label = dict_alph[img_name[0]]
         labels.append(label)
         img_train.append(im)
         
         data.append((im,label)) 

   return data, img_train, labels

In [9]:
train_data, imgs_train, labels_train = get_dataset(train_data_dir)
test_data, imgs_test, labels_test = get_dataset(testing_data_dir) 

./DatasetLIS/dataset1/train
./DatasetLIS/dataset1/testing


In [10]:
from torch.utils.data import DataLoader
train_data = DataLoader(train_data, batch_size=batch_size, num_workers=2, shuffle=True)
test_data = DataLoader(test_data, batch_size=batch_size, num_workers=2)

In [11]:
import torch
import torch.nn as nn
from torch.optim import SGD
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score
from os.path import join
import tqdm
import numpy as np

class AverageValueMeter:
    def __init__(self):
        self.reset()
        self.sum = 0
        self.num = 0

    def reset(self):
        self.sum = 0
        self.num = 0

    def add(self, val, n=1):
        self.sum += val*n
        self.num += n

    def value(self):
        try:
            return self.sum/self.num
        except:
            return None


def train_classifier(model, train_loader, test_loader, exp_name='experiment', lr=0.01, epochs=10, momentum=0.99, logdir='logs'):
    # optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
    criterion = nn.CrossEntropyLoss()
    writer = SummaryWriter(join(logdir, exp_name))
    loss_meter = AverageValueMeter()
    acc_meter = AverageValueMeter()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print("Device:", device)
    model.to(device)

    loader = {
        'train': train_loader,
        'test': test_loader
    }

    global_step = 0

    for e in range(epochs):
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            loss_meter.reset()
            acc_meter.reset()

            with torch.set_grad_enabled(phase=='train'):
                with tqdm.tqdm(enumerate(loader[phase]), total=len(loader[phase]), desc=f"{phase.capitalize()} Epoch {e+1}/{epochs}", unit="batch") as pbar:
                    for i, batch in pbar:
                        
                        x = batch[0].to(device)
                        y = batch[1].to(device)
                        output = model(x)

                        n = x.shape[0]
                        global_step += n
                        loss = criterion(output, y)

                        if phase == 'train':
                            loss.backward()
                            optimizer.step()
                            optimizer.zero_grad()
                            # scheduler.step(loss)

                        accuracy = accuracy_score(y.to('cpu'), output.to('cpu').max(1)[1])
                        loss_meter.add(loss.item(), n)
                        acc_meter.add(accuracy, n)

                        pbar.set_postfix(loss=loss_meter.value(), accuracy=acc_meter.value())

                        if phase == 'train':
                            writer.add_scalar('loss/train', loss_meter.value(), global_step=global_step)
                            writer.add_scalar('accuracy/train', acc_meter.value(), global_step=global_step)
            writer.add_scalar('loss/' + phase, loss_meter.value(), global_step=global_step)
            writer.add_scalar('accuracy/' + phase, acc_meter.value(), global_step=global_step)

        torch.save(model.state_dict(), '%s-%d.pth' % (exp_name, e+1))

    return model

def test_classifier_v2(model, loader):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    model.to(device)
    model.eval()

    predictions, labels = [], []

    with torch.no_grad():
        with tqdm.tqdm(loader, desc="Testing", unit="batch") as pbar:
            for batch in pbar:
                x = batch[0].to(device)
                y = batch[1].to(device)
                output = model(x)
                preds = output.to("cpu").max(dim=1)[1]
                labs = y.to("cpu").numpy()
                predictions.extend(list(preds))
                labels.extend(list(labs))

    return np.array(predictions), np.array(labels)

In [17]:
input_shape = (3, 64, 64) # Channels, Height, Width
model = SignLanguageCNN()

train_classifier(model, train_data, test_data, epochs=epochs, lr=0.001, logdir="Prima Prova")

test_result = test_classifier_v2(model, test_data)

Device: cuda


Testing: 100%|██████████| 20/20 [00:02<00:00,  6.90batch/s]


In [18]:
# print all test results in a for loop
counter = 0
for i in range(len(test_result[0])):
    if (sign_labels[test_result[0][i]] == sign_labels[test_result[1][i]]):
        counter += 1
    print("Predicted: ", sign_labels[test_result[0][i]], "Actual: ", sign_labels[test_result[1][i]])

print(counter / len(test_result[0]), " || ", counter, "/", len(test_result[0]))

Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  F Actual:  A
Predicted:  A Actual:  A
Predicted:  A Actual:  A
Predicted:  Y Actual:  A
Predicted:  Y Actual:  A


In [14]:
torch.save(model, 'bello_modello.pt')